Volgende stappen
- los probleem met JSON serializable op: SegmentedText is niet JSON serializable. DONE: subclass JSONEncoder.
- maak Flask based service, die weer asearch functies aanroept - DONE
- schrijf handler functies voor aservice REST calls, in de aservice package, op basis huidige functies

In [1]:
import sys
sys.path.append('../un-t-ann-gle')

In [2]:
from extractors import tei_extractor

path = '../data/tei-samples/bosb002graa04_01.xml'

(all_textelements, all_annotations) = tei_extractor.process([path])

In [3]:
all_textelements.len()

3130

In [7]:
# return generator for annotations with a specific label
#def get_annotations_of_type(type,annotations):
#    return (a for a in annotations if a['label'] == type)

from annotation import asearch

sample_para = [paras for paras in asearch.get_annotations_of_type('paragraph', all_annotations)][104]
print(sample_para['begin_anchor'])
print(sample_para['end_anchor'])
sample_para

anchor_f2cfd2d9-af95-4cf6-92d8-17fb499b5095
anchor_f2cfd2d9-af95-4cf6-92d8-17fb499b5095


{'label': 'paragraph',
 'begin_anchor': anchor_f2cfd2d9-af95-4cf6-92d8-17fb499b5095,
 'end_anchor': anchor_f2cfd2d9-af95-4cf6-92d8-17fb499b5095,
 'id': 'annot_bd77bac3-51b4-4526-bee3-b9941a84ba40'}

In [18]:
dir(all_textelements)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_anchors',
 '_new_anchor_id',
 '_ordered_segments',
 'append',
 'element_at',
 'extend',
 'len',
 'slice',
 'split']

In [8]:
all_textelements.slice(sample_para['begin_anchor'], sample_para['end_anchor'])

['Hij wilde iets antwoorden, maar zij luisterde niet. ‘Lieve Mary!’ zoo wendde zij zich tot de andere juffer, ‘zeg gij aan Sawers ons een beker seek te brengen en schuif die tabouret voor Mylord bij het vuur.’']

In [ ]:
begin_anchor = all_textelements.split(sample_para['begin_anchor'], 11)
end_anchor = all_textelements.split(begin_anchor, 8)

In [ ]:
all_textelements.slice(begin_anchor, begin_anchor)

In [ ]:
all_annotations.append({'label':'entity','begin_anchor': begin_anchor,'end_anchor': begin_anchor,\
                    'id': 'annot_'+str(uuid.uuid4()), 'entity_type': 'per', 'entity_text': 'Geertrui'})

In [ ]:
entities = [ents for ents in asearch.get_annotations_of_type('entity', all_annotations)]
entities

In [ ]:
# print all annotations overlapping with the entity
for a in asearch.get_annotations_overlapping_with(entities[0]['begin_anchor'],entities[0]['end_anchor'],all_annotations):
    print(a)

Gebruik bovenstaande voor het visualiseren van alle tekst voor een pagina, of alle heads (vanaf het midden van het boek)

In [ ]:
for a in asearch.get_annotations_of_type('page', all_annotations):
    if a['id'] == 'page-116':
        print(a)

In [ ]:
p116 = [p for p in asearch.get_annotations_of_type('page', all_annotations) if p['id'] == 'page-116'][0]

for t in all_textelements.slice(p116['begin_anchor'],p116['end_anchor']):
    print(f"{t}\n")

In [ ]:
import json

segtextJSONData = json.dumps(all_textelements, indent=4, cls=segmentedtext.SegmentEncoder)
print(segtextJSONData)

In [ ]:
# write all_textlines to a file
with open(datadir+'all_tei_textlines.json', 'w') as filehandle:
    json.dump(all_textelements, filehandle, cls=segmentedtext.SegmentEncoder)

In [ ]:
# write all_annotations to a file
with open(datadir+'all_annotations.json', 'w') as filehandle:
    json.dump(all_annotations, filehandle, cls=segmentedtext.AnchorEncoder)

In [ ]:
from annotation import aservice
from flask import Flask
import requests
import json

In [ ]:
headers = {'Content-Type': 'application/json'}
adata = json.dumps(all_annotations, cls=segmentedtext.AnchorEncoder)
response = requests.put("http://localhost:5000/annotations", data = adata, headers=headers)

In [ ]:
response = requests.get("http://localhost:5000")

In [ ]:
response.json()